# 💳 Finanzas y Banca: riesgo, fraude, rentabilidad
KPIs: NPL, PD/LGD/EAD, NIM, churn, CAC, fraude por canal.

## Casos de uso
- Scoring de crédito y límites dinámicos
- AML/KYC, detección de anomalías transaccionales
- Segmentación RFM y propensión a compra
- Monitoreo de liquidez y escenarios regulatorios (IFRS9/Basilea)

In [ ]:
# Mini ejemplo PD con features simples (toy)
def prob_default(score: float) -> float:
    import math
    z = -2.0 + 0.03 * score
    return 1/(1+math.exp(-z))

[round(prob_default(s),3) for s in [300,500,700]]

## Arquitectura
- Flujos transaccionales en streaming
- Feature store para variables de riesgo
- Modelos gobernados, explainability y trazabilidad
- Data contracts y auditoría regulatoria